# Un générérateur d'article

- Autair : Silanoc
- octobre 2023
- version : 2.1

Il s'agit d'un programme qui génére des articles de façon aléatoire. 

Pour cela, il analyse, dans des textes en entrée, la fréquence des mots qui se suivent.
Pour l'entrainer, les textes seront issus de la wikipedia francophone.

En sortie un texte aléatoire sera proposé en fonction de ces fréquences.

In [22]:
import os
import random
import ipywidgets as widgets
import wikipedia
from datetime import datetime

## Avoir un corpus de texte

### ajout manuel

On peut mettre en entrée des fichiers .txt directement dans le dossier qui servira de sources plus tard "vrai_texte" (par défaut).
Le chemin sera à mettre dans la varible "ou_sont_les_sources" de la partie "Enfin, générer notre article"

In [25]:
%%HTML
<div class='img_et_titre' style="display: flex ; flex-direction:row">
    <img src="./statics/wikipedia.png"/> 
    <h3 style="color:ff0">Avoir un corpus de texte, extraction de wikipedia.</h3>
</div>

Code générer avec l'aide de chat-GPT.
Il s'agit de sélectionner des articles de la version franophone de wikipedia (par défaut) et d'en extraire le texte sans les mises en forme.

#### Initialisation des variables

In [7]:
# Définir le nombre d'articles à extraire
# Attention, minimum 2 
nombre_articles_a_extraire :int = 2

# Où les sauver
#chemin_extraction : str =  "./vrai_texte/wikipedia/"
#chemin_extraction : str =  "./vrai_texte/hft/"
chemin_extraction : str =  "./vrai_texte/tout/"

# Choix de la version linguistique 
langue : str = "fr"

In [16]:
style = {'description_width': 'initial'}

choix_nb_article = widgets.BoundedIntText(
    value = 3, min = 2,max = 20, step = 1,
    description = "Combien d'articles voulez vous extraire ?", style = style, 
    disabled = False,
    continuous_update = False,
    readout = True,
    width = '25%', height = '160px',
    layout = widgets.Layout(width = '100%', height = '50px'))

display(choix_nb_article)

BoundedIntText(value=3, description="Combien d'articles voulez vous extraire ?", layout=Layout(height='50px', …

In [11]:
nombre_articles_a_extraire = choix_nb_article.value
nombre_articles_a_extraire

20

#### Le code d'extraction 

In [10]:
def extraction_wikipedienne (langue, chemin_extraction, nombre_articles_a_extraire) -> None :
    """Selectionne au hasard un nombre d'article de wikipédia en chaoissiassant sa langue.
    Pour chacun de ces articles, extraire le corps de l'article et le nettoyer de la syntaxe spécifique de wikipédia
    Mettre ces textes dans des fichiers txt dont le nom est celui de l'article
    """
    # Définir la langue
    wikipedia.set_lang(langue)  

    try:
        # Obtenir une liste aléatoire de titres d'articles
        titles : list[str] = wikipedia.random(nombre_articles_a_extraire)

        # Pour chaque article, extraire le contenu et enregistrer dans un fichier
        for title in titles:
            # Obtenir le contenu de l'article
            content = wikipedia.page(title).content

            # Supprimer les signes == ou === et double saut de ligne du contenu
            content = content.replace("===", "")
            content = content.replace("==", "")
            content = content.replace("\n\n", "")

            # Créer un fichier avec le titre de l'article comme nom
            file_name = chemin_extraction + title + ".txt"
            with open(file_name, "w", encoding="utf-8") as f:
                f.write(content)
            print(f"""Article {title} ajouté""")
    except:
        print(f"""l'article {title} génére une erreur""")

#### Extraction en elle même

In [11]:
extraction_wikipedienne(langue, chemin_extraction, nombre_articles_a_extraire)

Article Campyloptère des Santa Marta ajouté
Article Cour suprême de Gambie ajouté


## Gérer les fichiers

Pour se simplifier la vie (ou pas), tout ce qui touche à l'ouverture, lecture, écriture de fichiers est regroupé dans une classe

In [5]:
class Gestionfichier():
    """classe pour gérer les dossiers et fichiers"""
    
    def lister_fichier(self, source : str)-> list:
        """pour tous les fichiers du dossier, 
        on fait confiance qu'il ne s'agit que d'un texte,
        mettre tous les noms dans une liste
        arg
        - source (str) : chemin du dossier
        return
        - tousleschemins(liste) : liste avec les nom des image dans la source
        """
        tousleschemins: list = []
        for fichier in os.listdir(source):
            tousleschemins.append(chemin_extraction + fichier)
        return tousleschemins
 
    def lirefichier(self, chemin: str)-> str:
        """ouvre un fichier et transfert son contenue dans une chaine
        arg
        - chemin (str) : chemin du dossier
        return
        - contenu_du_fichier (str) : le contenu du fichier"""
        fichier = open(chemin, 'r')
        contenu_du_fichier: str = fichier.read()
        return contenu_du_fichier
    
    def ecrirefichier(self, chemin: str, nom: str, contenu: str)-> None:
        """ecrit un chaine dans un fichier"""
        with open(f"{chemin}/{nom}.txt", "w", encoding="utf-8") as fichier_sorti:
            fichier_sorti.write(contenu)
            fichier_sorti.close()
       

## Analyser des fichiers

### la classe Article_source

In [6]:
class Article_source():
    """chaque article qui servira d'entrée en apprentissage sera mis dans un objet pour analyse, extraction..."""
    
    def __init__(self, text):
        """initilisation
        arg: 
            une chaine de texte (si possible longue)
        return :
            aucun
        """
        self.texte:str = text #le texte sur lequel on travail
        self.dicostatique:dict = {} #pour la fonction liste_et_compte_mots
        self.dicodoublons:dict = {} #pour la fonction cherche_binomes_mots - ce qui est recherché
    
    def retirer_ponctuation(self, txt_a_nettoyer:str) -> str:
        """ La première version du logiciel est sommaire. 
        Pour se simplifier la vie, il faut supprimer tous les signe de ponctuations.
        arg:
            une chaine de texte avec de la ponctuation
        return:
            une chaine de texte sans ponctuation
        """
        ponctuation : list = [",",";",":","!","?",".","/","«","»",'"',"–","(",")"]
        for signe in ponctuation:
            txt_a_nettoyer = txt_a_nettoyer.replace(signe, "")
        #mettre un espace entre les mots avec apostrophe afin de bien les séparer
        apostrophe = ["’","'"]
        for apost in apostrophe:
            txt_a_nettoyer = txt_a_nettoyer.replace(apost, " ")
        #quand on supprime un : par exemple, cela fait deux espace. Remplacer ces artefacts de cagage
        for _ in range(len(self.texte)):
            txt_a_nettoyer = txt_a_nettoyer.replace("  ", " ")
        for _ in range(len(self.texte)):
            txt_a_nettoyer = txt_a_nettoyer.replace(" ", " ")
        return txt_a_nettoyer

    def liste_et_compte_mots(self, texte_a_traiter:str) -> list[list,dict]:
        """Compte le nombre d'occurence d'un mot.
        Fonction créée un peu par erreur, mais elle peut être utile pour faire des statistiques.
        arg : 
            texte_a_traiter (str)
        return : 
            2 object dans un liste. La liste des mots de la chaine et le comptrage de chaque mot dans un dictionnaire
            """
        liste_mot: list[str] = texte_a_traiter.split(" ")
        dicostatistique = {}
        for mot in liste_mot:
            if mot in dicostatistique:
                dicostatistique[mot] += 1
            else:
                dicostatistique[mot] = 1
        return [liste_mot, dicostatistique]
    
    def cherche_binomes_mots(self, texte_a_traiter:str) -> dict:
        """la fonction principale de l'objet : faire un dictionnaire de fréquences des mots qui se suivent. 
        arg :
            self
        return :
            dictionnaire {mot1:[mot2, mot3, mot3, mot4],...}
        """
        liste_mots_suivant:list = []
        dicodoublons:dict = {}
        #recherche des espaces délimitants les mots
        list_position_espace = []
        for i in range(len(texte_a_traiter)):
            if texte_a_traiter[i] == " ":
                list_position_espace.append(i)
        #print(list_position_espace)
        #recherche doublons mot
        debut1 = 0
        for i in range(len(list_position_espace)-2): #AFAIRE : ATTENTION ça ne prends pas les deux derniers mot. A vérifier.
            fin1 = list_position_espace[i]
            fin2 = list_position_espace[i + 1]
            #print(debut1, fin1,fin2)
            mot1 = texte_a_traiter[debut1:fin1]
            mot2 = texte_a_traiter[fin1:fin2]
            debut1 = list_position_espace[i]
            #print(mot1,mot2)
            liste_mots_suivant.append([mot1, mot2])
        #print(liste_mots_suivant)
        #fait un dictionnaire avec toutes les occurences possible après un même mot.
        #les doublons sont normaux, cela veut dire que le mot revient plusieurs fois, cela correspond au calcul de leur fréquence   
        for j in range(len(liste_mots_suivant)):
            if liste_mots_suivant[j][0] in dicodoublons.keys():
                #print('doublons')
                dicodoublons[liste_mots_suivant[j][0]].append(liste_mots_suivant[j][1])
            else:
                #print('nouveau')
                dicodoublons[liste_mots_suivant[j][0]]=[(liste_mots_suivant[j][1])]
        # Warning : il y a des espace en trop (tenu en compte pour le reste et tests)
        return dicodoublons 
    
    def tout_enchainer(self) -> dict :
        self.texte = self.retirer_ponctuation(self.texte)
        #self.dicostatique = self.liste_et_compte_mots(self.texte)[1]
        #print(self.dicostatique)
        #print("--------------")
        dicodoublons_txt = self.cherche_binomes_mots(self.texte)
        return dicodoublons_txt

### Tester les méthodes de la classe Article_source

In [7]:
#!pip install pytest

In [8]:
import pytest
from unittest.mock import Mock, patch 

In [9]:
class Test_modele_analyse():
    
    def test_retirer_ponctuation(self):
        chaine = "Le, petit chat d'Hercule est mort !"
        textdebase = Article_source(chaine)
        retirer = textdebase.retirer_ponctuation(chaine)
        assert retirer == "Le petit chat d Hercule est mort "
        
    def test_liste_et_compte_mots(self):
        chaine = "le petit chat de béatrice est sur le petit mur du jardin de Yves"
        textdebase = Article_source(chaine)
        list_mot = textdebase.liste_et_compte_mots(chaine)[0]
        dico_mot = textdebase.liste_et_compte_mots(chaine)[1]
        assert list_mot == ['le','petit','chat','de','béatrice','est','sur','le','petit','mur','du','jardin','de','Yves']
        assert dico_mot == {'le':2,'petit':2,'chat':1,'de':2,'béatrice':1,'est':1,'sur':1,'mur':1,'du':1,'jardin':1,'Yves':1}
     
    """  
    #code pour tester une erreur dans jupyter 
    def test_cherche_binomes_mots(self):
        chaine = "le petit chat de béatrice est sur le petit mur du jardin de Yves"
        textdebase = Article_source(chaine)
        dico = textdebase.cherche_binomes_mots(chaine)
        assert dico == {' le': [' petit', ' petit'], ' petit':[' chat', ' mur'], ' chat': [' de'], ' de': [' béatrice', ' Yves'], ' béatrice': [' est'],
                        ' est': [' sur'], ' sur': [' le'], ' mur': [' du'], ' du': [' jardin'], ' jardin': [' de'], ' Yves': []}
    """ 

In [10]:
testeur = Test_modele_analyse()
testeur.test_retirer_ponctuation()
testeur.test_liste_et_compte_mots()

## Générer du texte

### La classe Article_au_hasard

In [12]:
class Article_au_hasard():
    """génére un texte aléatoire à partir d'un dictionnaire
    
        arg :
            dictionnaire {mot1:[mot2, mot3, mot3, mot4],...}
            typiquement le dictionnaire généré par le return de cherche_binomes_mots(), ou d'une sauvegarde issus de cette fonction.
        
        return
            une chaine de texte avec les mots du dictionnaire dans un ordre aléatoire.
            elle pourra aller dans un doc txt pour sauvegarde
    """

    def __init__(self, mondico:dict):
        """initilisation
        arg: 
            mondico (dict) : pour chaque mot en cle dedans, il y a une liste de mot possible.
        """
        self.mondico: dict = mondico #le dictionnaire sur lequel on travail
        self.textealeatoire : str ="" #le texte que l'on veut
    
    def choixmotpourcommencer(self, dico: dict)-> str:
        """a utiliser pour le premier mot, 
        mais aussi si un mot ne peut pas en trouver d'autre, faire une proposition pour eviter une erreur et continuer
        """
        #Mettre les clefs dans une liste
        liste_des_mots : list = []
        for key in dico:
            liste_des_mots.append(key)
        #choix lui meme
        mot: str = liste_des_mots[random.randint(0, len(liste_des_mots))]
        #retirer l'espace s'il existe
        if mot[0] == " ":
            mot = mot[1:]
        return mot      

    def chercherlemotsuivant(self, dico: dict, mot: str) -> str:
        """à partir d'un mot, sortir aléatoire un mot dans ceux pouvant le suivre stocké dans le dictionnaire"""
        liste_des_possible: list = dico[" " + mot] #WARNING on remet un espace car dans la version du moment, il y a un espace dans le dico et c'est pas bien
        mot: str = liste_des_possible[random.randint(0, len(liste_des_possible) -1 )]
        #retirer l'espace s'il existe
        if mot[0] == " ":
            mot = mot[1:]
        return mot      
    
    def genereruntexte(self, taille_article: int) -> None:
        """intier avec choixmotpourcommencer(), puis enchainer chercherlemotsuivant()
        taille_article est le nombre de mot que l'on veut pour l'article aléatoire
        """
        mot: str = self.choixmotpourcommencer(self.mondico)
        self.textealeatoire += mot
        for _ in range(taille_article - 1):
            new_mot: str = self.chercherlemotsuivant(self.mondico, mot)
            mot = new_mot
            self.textealeatoire = self.textealeatoire + " " + mot

### Tester la class Article_au_hasard

In [13]:
class Test_modele_generation():
    
    def test_choixmotpourcommencer(self, mocker):
        dictionnaire = {' le': [' petit', ' petit'], ' petit':[' chat', ' mur'], ' chat': [' de'], ' de': [' béatrice', ' Yves'], ' béatrice': [' est'],
                        ' est': [' sur'], ' sur': [' le'], ' mur': [' du'], ' du': [' jardin'], ' jardin': [' de'], ' Yves': []}
        mocker.patch('random.randint', return_value=1)
        generation = Article_au_hasard(dictionnaire)
        generation.mot = generation.choixmotpourcommencer(dictionnaire)
        assert generation.mot == "petit"

In [14]:
testeur_article = Test_modele_generation()
testeur_article.test_choixmotpourcommencer(None)


AttributeError: 'NoneType' object has no attribute 'patch'

## Gestion de dictionnaire

### Additioner des dictionnaires

In [15]:
def addition_dico(grand_dico : dict, petit_dico : dict)-> dict:
    """Permet de mettre le contenu d'un dictionnaire dans un autre.
    Si dans les deux dic, il y a une clef commune, il concatène les deux listes de mot
    """
    for keys, values in petit_dico.items():
        if keys in grand_dico:
            liste_intermediaire = grand_dico[keys]
            for item in values:
                liste_intermediaire.append(item)
            grand_dico[keys] = liste_intermediaire
        else:
            grand_dico[keys] = values
    return grand_dico

### tester l'addition de dictionnaire

In [16]:
def test_addition_dico():
    dico1 = {'a' : ['le', 'petit'], 'b' : ['chat', 'de']}
    dico2 = {'b' : ['de', 'yves'], 'c' : ['dans', 'jardin']}
    dico3 = addition_dico(dico1, dico2)
    assert dico3 == {'a' : ['le', 'petit'], 'b' : ['chat', 'de', 'de', 'yves'], 'c' : ['dans', 'jardin']}
    
test_addition_dico()

### Analyser tous les fichiers d'un dossier et en faire un dictionnaire

In [17]:
def analyser_dossier_et_faire_dico(source : str) -> dict:
    """Pour tous les fichiers d'un dossier, 
    lire le contenue, en faire le dictionnaire 
    et fusionner tous les dictionnaire en un seul.
    arg:
    - source (str) : chemin d'un dossier
    return:
    - dict_statistique (dict)
    """
    gestionnaire = Gestionfichier()
    lesfichichiers = gestionnaire.lister_fichier(source)
    #print(lesfichichiers)
    dict_statistique = {}
    for item in lesfichichiers:
        contenu_a_analyser = gestionnaire.lirefichier(item)
        analyseur = Article_source(contenu_a_analyser)
        dico_item = analyseur.tout_enchainer()
        dict_statistique = addition_dico(dict_statistique, dico_item)
    return dict_statistique

## Enfin, générer notre article

A partir des fichiers de 'vrai_texte/wikipedia', génére un texte de 250 mots.

In [18]:
#ou_sont_les_sources = "vrai_texte/wikipedia"
#ou_sont_les_sources = "vrai_texte/hft"
ou_sont_les_sources : str = "vrai_texte/tout"
nb_mot_article : int = 250

In [18]:
style = {'description_width': 'initial'}

choix_nb_mot = widgets.BoundedIntText(
    value = 250,min = 100,max = 1000, step = 1,
    description = "Combien de mot doit faire l'article ?", style = style, 
    disabled = False,
    continuous_update = False,
    readout = True,
    width = '25%', height = '160px',
    layout = widgets.Layout(width = '100%', height = '50px'))

display(choix_nb_mot)

BoundedIntText(value=250, description="Combien de mot doit faire l'article ?", layout=Layout(height='50px', wi…

In [11]:
nb_mot_article = choix_nb_mot.value
nb_mot_article

1000

In [19]:
# préparer le dictionnaire qui pourra être réutilisé plusieurs fois
superdico = analyser_dossier_et_faire_dico(ou_sont_les_sources)

In [19]:
def generation() -> None :
    redacteur = Article_au_hasard(superdico)
    redacteur.genereruntexte(nb_mot_article)
    article_aleatoire_redige = redacteur.textealeatoire
    return article_aleatoire_redige

In [20]:
# Cellule à exécuter pour un nouveau texte
article_obtenue = generation()
print(article_obtenue)

NameError: name 'Article_au_hasard' is not defined

In [22]:
# Pour horodater le fichier
def horodatage():
    now = datetime.now()
    timestamp = now.strftime("%Y%m%d%H%M%S%f") #A la micro-seconde prêt 
    return timestamp
    # print(timestamp)

In [23]:
def avoir_un_fichier()-> None:
    ecrivain = Gestionfichier()
    ecrivain.ecrirefichier("./textes_generes", str(horodatage()), article_obtenue)

In [24]:
avoir_un_fichier()